# Day 11 - 2D Cellular automatons

* https://adventofcode.com/2020/day/11

This is familiar territory for me, as we covered similar systems in years before (see days [12](../2018/Day%2012.ipynb) and [18](../2018/Day%2018.ipynb) in 2018, and days [11](../2019/Day%2011.ipynb) and [24](../2019/Day%2024.ipynb) in 2019).

Like before, using numpy, [`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy-1.5.4/reference/generated/scipy.signal.convolve2d.html), and [`numpy.select()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html) make it really easy to generate each state.

I actually discovered a bug in my original Day 18 code: I had _transposed_ the input map. That was fine then, but for this puzzle I couldn't figure out why my puzzle would not converge until I minutely compared my puzzle input with the matrix produced. Oops.


In [1]:
from enum import Enum
from scipy.signal import convolve2d
import numpy as np

class Seat(Enum):
    floor = 0, '.'
    empty = 1, 'L'
    occupied = 2, '#'
    
    def __new__(cls, int_: int, value: str):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.int = int_
        return instance

_kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])

class WaitingArea:
    def __init__(self, waiting_area_map: str) -> None:
        self._matrix = np.array([
            Seat(c).int for line in waiting_area_map.splitlines() for c in line
        ]).reshape((-1, waiting_area_map.index('\n')))

    def __str__(self) -> str:
        mapping = {s.int: s.value for s in Seat}
        return '\n'.join([''.join(map(mapping.__getitem__, row)) for row in self._matrix])

    @property
    def occupied(self) -> int:
        return np.sum(self._matrix == Seat.occupied.int)

    def run(self) -> int:
        """Run until stability is reached, the return the number of occupied seats"""
        f = self._matrix
        full = {s: np.full(f.shape, s.int) for s in Seat if s is not Seat.floor}
        while True:
            counts = {
                seat: convolve2d(f == seat.int, _kernel, mode='same')
                for seat in Seat
                if seat is not Seat.floor
            }
            rules = {
                # If a seat is **empty** (L) and there are **no** occupied seats adjacent to
                # it, the seat becomes **occupied**.
                Seat.occupied: (f == Seat.empty.int) & (counts[Seat.occupied] == 0),
                # If a seat is **occupied** (#) and **four or more** seats adjacent to it are
                # also occupied, the seat becomes **empty**.
                Seat.empty: (f == Seat.occupied.int) & (counts[Seat.occupied] >= 4),
            }
            f = np.select(list(rules.values()), [full[s] for s in rules], default=f)
            if np.array_equal(self._matrix, f):
                return self.occupied
            self._matrix = f

test_map = """\
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
assert WaitingArea(test_map).run() == 37

In [2]:
import aocd
area_map = WaitingArea(aocd.get_data(day=11, year=2020))

In [3]:
print("Part 1:", area_map.run())

Part 1: 2438
